In [172]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from scipy import stats
import seaborn as sns
import re

import tables as tb

import pickle
import requests
import geocoder
from bs4 import BeautifulSoup
from collections import*
import matplotlib
import matplotlib.pyplot as plt
from PythonSrc.hdf5_getters import *

import PythonSrc.hdf5_getters as GETTERS


import os
import sys
import time
import glob
import datetime
import sqlite3

import unidecode

%matplotlib inline

### Create a dictionary:
This dictionary should match the maximum of those list of terms to a genre:
- `subset_unique_terms.txt`
- `subset_unique_mbtags.txt`

Define what is a genre : First dictionnary used the data from the website: https://www.quora.com/What-is-the-basic-difference-between-different-genres-of-music-jazz-rock-pop-blues-rap-and-so-on

In [24]:
#LIST YOU SHOULD GET AFTER SCRAPING
quora_list=[['Blues' ,'Blues Rock', 'Chicago Blues', 'Delta Blues','Electric Blues ', 'Jump Blues', 
       'New OrleansBlues', 'St Louis Blues', 'Swamp Blues', 'Texas Blues', 'Zydeco', 'Gospel'],
       ['Country', 'Bluegrass', 'Country Pop', 'Honky Tonk', 'Mountain' ,'Neo traditional',
        'Outlaw', 'Western'],['Electronica','Electro','Ambient','Dubstep','House','Techno','Trance','TripHop'],
        ['Folk','American Roots' ,'Folk pop','Folk rock', 'jam'],['Jazz', 'acid jazz','be bop',
         'big band', 'jazz funk', 'jazz fusion','latin jazz', 'ragtime', 'smooth jazz','soft jazz',
        'tarditional jazz', 'voca jazz'],['Metal', 'Avant Garde Metal', 'Black Metal', 'Classic metal',
        'death metal','doom metal','funk metal','glam metal','goth metal','Industrial metal','jazz metal',
        'metalcore', 'neoclassical metal', 'nu metal', 'power metal', 'progressive metal', 'rap metal',
        'sleaze metal','sludge metal', 'speed metal', 'symphonic metal','trash metal','viking metal'],
        ['Orchestral', 'Broadway', 'Classical', 'movie scores', 'new age', 'vocal'],['Pop',
        'AdultContemporary', 'beat pop', 'dance pop', 'disco','doowop','indie pop','latin pop', 'pop rock',
        'power pop','surf pop', 'synth pop','traditional pop'], ['Hiphop','R&B','rnb','Alternative Hip Hop',
        'Funk','neosoul', 'new jack swing', 'rap','soul','urban contemporary'],['Reggae','Dancehall','dub','raga',
        'roots'],['Rock','Alternative rock','americana','artrosi','classic rock','college rock','darkwave','funkrock',
        'grunge','hardrock','indierock','new wave post grunge','pogressive rock','psychedelic rock','punk',
        'rockabilly','ska','soft rock','southernrock'],['World','African','Middle Eastern', 'Asian','Eastern european',
        'Irish','celtic','gaelic','island','latin','hispanic','mediterranean','Scandinavian']]

TODO: 
- Add: hiphop, electro, elctronic, opera,  rnb to the list, change the name of rnb to hiphop.
      
- take care during scraping while thre is a `/` to  separate the words: `'latin/hispanic'-->'latin','hispanic'`

**Create a function clean_words** to have the terms in the same format:
- without numbers
- without special carters !?-/;
- without capital letters
- without accent
- without space a the begining of the string and at the end, and just one space between words
- we also change the word 'hip hop' to 'hiphop' as it's a common words.


In [25]:
def clean_word(w):

    w=re.sub(r'[^\w+]',' ', w)  #remove carater
    w=re.sub(r'[\b\d+\b]',' ', w) #remove number
    w=w.lower()     # change capital letter to lower ones
    w=unidecode.unidecode(w) #remove accent
    w=' '.join(w.split()) #remove double spaces and spaces at the biginig and at the end
    w=re.sub(r'hip hop','hiphop', w) # change 'hip hop' to 'hiphop'
  
    return w

In [26]:
# example of the cleaning done by clean_word()
clean_word(' --> 90s    Hip hop!!   ')

's hiphop'

** Create the dictionnary **  here take the first word of the each sub_list of the `quora_list` as the category a

In [27]:
my_dict={}
for sl in quora_list:
    categorie=clean_word(sl[0])
    for sub in sl:
        my_dict[clean_word(sub)]=categorie
        

In [28]:
#display the first dictionnary obtained by quora : we have 134 in our dictionnary
print(len(my_dict))
my_dict

134


{'acid jazz': 'jazz',
 'adultcontemporary': 'pop',
 'african': 'world',
 'alternative hiphop': 'hiphop',
 'alternative rock': 'rock',
 'ambient': 'electronica',
 'american roots': 'folk',
 'americana': 'rock',
 'artrosi': 'rock',
 'asian': 'world',
 'avant garde metal': 'metal',
 'be bop': 'jazz',
 'beat pop': 'pop',
 'big band': 'jazz',
 'black metal': 'metal',
 'bluegrass': 'country',
 'blues': 'blues',
 'blues rock': 'blues',
 'broadway': 'orchestral',
 'celtic': 'world',
 'chicago blues': 'blues',
 'classic metal': 'metal',
 'classic rock': 'rock',
 'classical': 'orchestral',
 'college rock': 'rock',
 'country': 'country',
 'country pop': 'country',
 'dance pop': 'pop',
 'dancehall': 'reggae',
 'darkwave': 'rock',
 'death metal': 'metal',
 'delta blues': 'blues',
 'disco': 'pop',
 'doom metal': 'metal',
 'doowop': 'pop',
 'dub': 'reggae',
 'dubstep': 'electronica',
 'eastern european': 'world',
 'electric blues': 'blues',
 'electro': 'electronica',
 'electronica': 'electronica',
 '

enrich our dictionnary, on the first list `subset_unique_terms.txt`,
we check for each term of this list if they are already present on our dictionnar.
If not we check each word of the term separetly (starting from the last one), if they belong to our dictionnary. If yes we add the entire terms with the found category.

For example: `'lovers rock'` doesn't belong to our dictionary so we check the word separatly as `'rock'` belong to  our dictionnary, we add   `my_dict['lovers rock']='rock'`

*here we printed the word, that are not catch in our dictionnary*

In [29]:
path='MillionSongSubset/AdditionalFiles/subset_unique_terms.txt'
terms=  open(path, 'r') 

original_in_dict=0
add_in_dict=0
not_in_dict=0


for t in terms:
 
    match=False
    clean_t=clean_word(t)
    split_t=clean_t.split()
    if clean_t not in my_dict:
        if split_t[-1] in my_dict:
            categorie=my_dict[split_t[-1]]
            my_dict[clean_t]=categorie
            #print('Match:',clean_t,categorie)
            add_in_dict+=1
            match=True
        else:
            for w in split_t[:-1]:
                if w in my_dict:
                    categorie=my_dict[w]
                    my_dict[clean_t]=categorie
                    #print('match:',clean_t,categorie)
                    add_in_dict+=1
                    match=True
                    break
    else:
        match=True
        original_in_dict+=1
        
    if match==False:
        not_in_dict+=1
        print('NO :',clean_t)
        
                
                
                

NO : s
NO : s
NO : s
NO : th century
NO : th century opera
NO : meter sessies
NO : step
NO : tone
NO : bonnaroo
NO : th century
NO : st century
NO : s
NO : s
NO : watt club
NO : ad
NO : s
NO : s lounge
NO : s
NO : s garage
NO : s girl group
NO : s influenced
NO : s psychedelic and garage
NO : s revival
NO : s spanish
NO : s
NO : rpm
NO : bit
NO : s
NO : s goth
NO : s hardcore
NO : s underground
NO : s
NO : s dance
NO : a cappella
NO : aaa
NO : aacm
NO : aberdeen
NO : aboriginal
NO : abstract
NO : accordion
NO : achingly intelligent
NO : acid
NO : acousmatic
NO : acoustic
NO : acoustic guitar
NO : acustica
NO : adagio
NO : adoracao
NO : adult alternative
NO : adult contemporary
NO : africa
NO : africosmic
NO : afrikan tahdet
NO : afro
NO : afro cuban
NO : afrobeat
NO : afropop
NO : afterdark
NO : aftermath
NO : aggressive
NO : aggrotech
NO : alabama
NO : alberta
NO : all female
NO : alliance
NO : alternative
NO : alternative s
NO : alternative dance
NO : alternative glam
NO : alternativ

NO : ironic
NO : isf
NO : iskelma
NO : israel
NO : italian
NO : italian dance
NO : italian indie
NO : italian music
NO : italian opera
NO : italophilia
NO : italy
NO : ithaca
NO : ivory coast
NO : jacksonville
NO : jah
NO : jakarta
NO : jamaica
NO : jamaican
NO : jamendo
NO : jangle
NO : jangly
NO : japan
NO : japanese
NO : japanese hardcore
NO : japanese indie
NO : japanese music
NO : japanoise
NO : java
NO : jawaiian
NO : jazzclub eisenach
NO : jazzdance
NO : jazzstep
NO : jersey
NO : jewish
NO : jewish music
NO : jig
NO : jive
NO : joululaulut
NO : jrock
NO : judaica
NO : jug band
NO : juggalo
NO : jump
NO : jump up
NO : jumpstyle
NO : jungle music
NO : k records
NO : kahvi
NO : kalimba
NO : kansas
NO : kansas city
NO : karaoke
NO : katzenmusik
NO : kazakhstan
NO : kazoo
NO : kbd
NO : kentucky
NO : kenya
NO : kenyan
NO : keyboard
NO : kids
NO : kingston
NO : kirtan
NO : kizomba
NO : klezmer
NO : knoxville
NO : kompakt label
NO : korean
NO : korean indie
NO : kpop
NO : kraut
NO : kri

NO : santa rosa
NO : santo domingo
NO : sao paulo
NO : sarah records
NO : saskatchewan
NO : satanic
NO : satire
NO : saxophone
NO : scandinavia
NO : scat
NO : schlageroldies
NO : schranz
NO : sci fi
NO : score composer
NO : scotland
NO : scottish
NO : scottish indie
NO : scratch
NO : scream
NO : screamo
NO : screamo hardcore
NO : search
NO : seattle
NO : seattle music
NO : senegal
NO : senegalese
NO : sensation black
NO : sensual
NO : sentimental
NO : serbia
NO : serialism
NO : sevilla
NO : sexy
NO : sexy female vocals
NO : shaman
NO : shana
NO : sharp
NO : sheffield
NO : sheffield bands
NO : shibuya kei
NO : shoegaze
NO : show tunes
NO : shred
NO : side project
NO : simple
NO : sing along
NO : singapore
NO : singaporean
NO : singer
NO : singer songwriter
NO : single
NO : skacore
NO : skane
NO : skate
NO : skiffle
NO : skinhead
NO : skinhead music
NO : skinhead reaggae
NO : skram
NO : slack key guitar
NO : slap bass
NO : sleaze
NO : sleaze glam
NO : slick
NO : slide
NO : slide guitar
N

In [30]:
print(' #terms that were already in our first dictionnary:', original_in_dict)
print(' #terms we added in our dictionnary :',add_in_dict)
print(' #terms we did not catch',not_in_dict)

 #terms that were already in our first dictionnary: 116
 #terms we added in our dictionnary : 1153
 #terms we did not catch 2233


We now do the same thing on the second list

In [31]:
path='MillionSongSubset/AdditionalFiles/subset_unique_mbtags.txt'
terms=  open(path, 'r') 

original_in_dict=0
add_in_dict=0
not_in_dict=0


for t in terms:

 
    match=False
    clean_t=clean_word(t)
    split_t=clean_t.split()

    if clean_t not in my_dict:
        if len(split_t)>0:
            if split_t[-1] in my_dict:
                categorie=my_dict[split_t[-1]]
                my_dict[clean_t]=categorie
                #print('Match:',clean_t,categorie)
                add_in_dict+=1
        
                match=True
            else:
                for w in split_t[:-1]:
                    if w in my_dict:
                        categorie=my_dict[w]
                        my_dict[clean_t]=categorie
                        #print('match:',clean_t,categorie)
                        add_in_dict+=1
                        match=True
                   
                        break
    else:
        match=True
        original_in_dict+=1
   
        
    if match==False:
        not_in_dict+=1
        print('NO :',clean_t)
        
                
                

NO : s
NO : 
NO : 
NO : s
NO : s
NO : s
NO : s
NO : s
NO : s
NO : 
NO : 
NO : ad
NO : s
NO : s
NO : s
NO : mile
NO : s
NO : s synthpop
NO : s
NO : a cappella
NO : ace of base
NO : acoustic
NO : actor
NO : adam levine
NO : adult contemporary
NO : akron
NO : alternative
NO : alternative dance
NO : american
NO : american idol
NO : american underground
NO : amherst
NO : amy lee
NO : americain
NO : anarcho
NO : angry
NO : angsty
NO : archives
NO : argentina
NO : argentine
NO : argentinian
NO : arranger
NO : arrangeur
NO : artist
NO : athens
NO : atlantic
NO : australia
NO : australian
NO : austria
NO : austrian
NO : austropop
NO : avant garde
NO : avante garde
NO : avantgarde
NO : b f lm
NO : bajofondo
NO : band
NO : barbade
NO : barbadian
NO : barbadien
NO : barbados
NO : bass
NO : belgian
NO : belgium
NO : ben gibbard
NO : ben moody
NO : berlin
NO : betulio medina
NO : bhangra
NO : big beat
NO : birmingham
NO : bob marley
NO : bobby valentin
NO : bolero
NO : bosnia and herzegovina
NO : bo

In [32]:
print(' #terms that were already in our second dictionnary:', original_in_dict)
print(' #terms we added in our dictionnary :',add_in_dict)
print(' #terms we did not catch',not_in_dict)

 #terms that were already in our second dictionnary: 210
 #terms we added in our dictionnary : 49
 #terms we did not catch 467


In [33]:
#final lenght of our dictionnary
print(len(my_dict))

1336


**GET GENRE**

get_genre1: return the genre,and the weight of the term with the maximum weight
get_genre2: for each term of the list `get_artist_terms` add the weight to the corresponding the genre 
            return teh genre with the maximum cumulative weight.

In [34]:
def get_genre1(h5,song_idx):
    
    for t_idx,t in enumerate(GETTERS.get_artist_terms(h5,song_idx)):
        t=t.decode("utf-8")
        clean_t=clean_word(t)
        if clean_t in my_dict:
            weight=GETTERS.get_artist_terms_weight(h5)[t_idx]
            return my_dict[clean_t],weight
        
    print(GETTERS.get_artist_terms(h5,song_idx),'\n')
    
    return 0,0

def get_genre2(h5,song_idx):
    #add a threshold
    genre_weight={'blues':0,'country':0,'electronica':0,'folk':0,'jazz':0,'pop':0,'metal':0,'rock':0,'orchestral':0,'hiphop':0,'reggae':0,'world':0}
    match=False
    
    for t_idx,t in enumerate(GETTERS.get_artist_terms(h5,song_idx)):
        t=t.decode("utf-8")
        clean_t=clean_word(t)
        if clean_t in my_dict:
            match=True
            weight=GETTERS.get_artist_terms_weight(h5)[t_idx]
            genre_weight[my_dict[clean_t]]+=weight
     
    if match==False:
        print(GETTERS.get_artist_terms(h5,song_idx),'\n')
        return 0,0
    else:
        
        genre=max(genre_weight, key=lambda k: genre_weight[k])
        #print(genre,'\n',GETTERS.get_artist_terms(h5,song_idx),'\n',genre_weight,'\n')
        return genre,genre_weight[genre]



**GET LOC**

Here return just the latitude and longitude -> TODO improve when you have the location-> transform to latitude and longitude

In [134]:
def get_loc(h5):
    lat = GETTERS.get_artist_latitude(h5)
    lng = GETTERS.get_artist_longitude(h5)
    location = GETTERS.get_artist_location(h5).decode("utf-8")
    
    return lat,lng,location    

In [177]:
df.head(10)

,genre,genreweight,year,duration,hotttnesss,familiarity,latitude,longitude,location
0,hiphop,6.498262,0,218.93179,0.401998,0.581794,34.052230,-118.243680,California - LA
1,rock,4.344685,1969,148.03546,0.417500,0.630630,35.149680,-90.048920,"Memphis, TN"
2,pop,0.794364,0,177.47546,0.343428,0.487357,NaN,NaN,
3,rock,6.624143,1982,233.40363,0.454231,0.630382,51.506420,-0.127210,"London, England"
4,rock,9.095158,2007,209.60608,0.401724,0.651046,NaN,NaN,
5,blues,1.660828,0,267.70240,0.385471,0.535293,NaN,NaN,
6,hiphop,4.373716,0,114.78159,0.261941,0.556496,40.413057,-82.711215,Ohio
7,rock,1.076524,0,189.57016,0.605507,0.801136,43.326200,-79.798630,"Burlington, Ontario, Canada"
8,rock,5.432989,1984,269.81832,0.332276,0.426668,NaN,NaN,
9,electronica,3.173096,0,266.39628,0.422706,0.550514,NaN,NaN,


In [182]:
def getLocation(row):
    if np.isnan(row.latitude) and row.location != "":
        
        print(row.location)
        g = geocoder.arcgis(row.location)
        try:
            lat,lng = g.latlng
        except:
            print("except")
            lat,lng = np.nan,np.nan
            print(lat)
        return pd.Series({'latitude':lat,'longitude':lng})
    else:
        return pd.Series({'latitude':row.latitude,'longitude':row.longitude})

**create_data_frame**

given the path of the millliong song data folder,
load each h5 files and for each sng of this h5 files, extract:
    
- The genre (with get_genre)
- the weight of genre
- the year, todo transform year to a pd.date? 
- the location (latitude, longitude)
- the duration of the song
- Familiarity – this corresponds to how well known in artist is.  You can look at familiarity as the likelihood that any person selected at random will have heard of the artist.  Beatles have a familiarity close to 1, while a band like ‘Hot Rod Shopping Cart’ has a familiarity close to zero.
- Hotttnesss – this corresponds to how much buzz the artist is getting right now. This is derived from many sources, including mentions on the web, mentions in music blogs, music reviews, play counts, etc.

https://musicmachinery.com/tag/hotttnesss/

In [142]:
def create_data_frame(data_path):
    columns = ['genre','genreweight','year','duration','hotttnesss','familiarity','latitude','longitude','location']
    dataset=pd.DataFrame(columns=columns)

    for root, dirs, files in os.walk(data_path):
        
        files = glob.glob(os.path.join(root,'*'+'.h5'))
        
        for f in files:
                
            h5 = GETTERS.open_h5_file_read(f)
            
            for song_idx in range(GETTERS.get_num_songs(h5)): 
                
                year = GETTERS.get_year(h5,song_idx)   
                lat,lng,location = get_loc(h5)
                genre,weight = get_genre2(h5,song_idx)
                hotttnesss = GETTERS.get_artist_hotttnesss(h5,song_idx)
                familiarity = GETTERS.get_artist_familiarity(h5,song_idx)
                duration = GETTERS.get_duration(h5,song_idx)
                
                if genre!=0:
                    song_df=pd.DataFrame([[genre,weight,year,duration,hotttnesss,familiarity,lat,lng,location]],columns=columns)         
                    dataset=dataset.append(song_df,
                                           ignore_index=True)#add index             
            h5.close()
            
        
    return dataset

In [143]:
# path to the Million Song Dataset code
msd_subset_path='MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check

In [160]:
cache_path = "cache/"
def get_data(name_file):
    '''Create and cache JSON data, return as a dataframe.'''
    path = cache_path + name_file
    try:        
        f = open(path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(path))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df = create_data_frame(msd_subset_data_path)
        df.to_pickle(path)
        print('Cached {} at {}'.format(path))
    return df

def save_data(df,name_file):
    path = cache_path + name_file
    df.to_pickle(path)

In [186]:
df = get_data("df1")
#save_data(df,"df1")

Loaded cache/df1 from cache


In [187]:
print('On 10000 samples we kept: ', len(df))

On 10000 samples we kept:  9754


In [188]:
# Very long to process
df.loc[:,['latitude','longitude']] = df.apply(lambda address: getLocation(address),raw=True,axis=1)

California - LA
California - LA
London, England
Ohio
Burlington, Ontario, Canada
Wisner, LA
Beverly Hills, CA
Houston, TX
New York, NY
South Jamaica, Queens
Middlesbrough, England
Garnant, Wales
Berkeley, CA
Pasadena, CA
Los Angeles, CA
Minnesota
CA
Bézu, comme Superdupont, ne connait qu&#039;un pays : la France !
Newark, NJ
Tallahassee, FL
Menilmontant, France
SPRINGFIELD, Virginia
Medicine Lodge, KS
Philadelphia, PA
Sydney, Australia
New York City
 Ubá, Minas Gerais
Seattle, WA
Barranquilla, Colombia
California - SF
Severn, MD
Locust Ridge, TN
Los Angeles, CA
Atlanta, GA
Compton CA
McLellan, FL
Los Angeles, CA
Oslo, Norway
Athens, GA
Fairfax, California
LDN
Poza Rica, MX
Cleveland, OH
Los Angeles, CA
Chicago
Bakersfield, CA
Atlanta, GA
Dublin, Ireland
Bakersfield, CA
London, England
Minneapolis, MN
McLellan, FL
Windhoek
Kitee, Finland
Burlington, Ontario, Canada
England
Steubenville, OH
Los Angeles, CA
Las Vegas, NV
Brookyln, NY
California - LA
Seattle, Washington USA
Gainesville, FL

Switzerland
Los Angeles &amp; New York
Switzerland
Birmingham, England
Chicago, IL
台北, Taiwan
Batley, England
Bakersfield, CA
Georgia
DURHAM, North Carolina
Atlanta, GA
New Jersey
ORANGE, California
Atlanta, GA
Nashville, TN
Syracuse, NY
California - LA
Tennessee
Middlesbrough, England
New York, NY
Newcastle
Memphis, Tennessee
California
Kingston, Jamaica
Gilmer, TX
Detroit MI
Chicago, IL
Wisconsin
California - LA
Massachusetts
Bézu, comme Superdupont, ne connait qu&#039;un pays : la France !
New York, NY [Harlem]
Melbourne, Australia
Pasadena, CA
London, England
ITALY
Seattle, WA
Sydney, Australia
Jackson MS
Tennessee
London, England
GERMANY
Atlanta, GA
Cross Lane, WV
ORANGE, California
United States
Ohio
Los Angeles, CA
Kingston, Jamaica
Menilmontant, France
Orange County, CA
M�engladbach, Germany
Kingston, Jamaica
Washington
Westbury, Long Island, NY
Carle Place, New York
NY - Long Island
Merida, Mexico
Geneva, NY
Boston, MA
Newark, NJ
Brooklyn, NY
San Francisco, CA
Wilmette, IL
Kin

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


except
nan
Sacramento, CA
Syracuse, NY
Barranquilla, Colombia
POLAND
Hollywood, CA
CANADA - Ontario
Poway, CA
Kingston, Jamaica
Minneapolis, MN
Oxford, UK
Oslo, Norway
San Francisco
Barranquilla, Colombia
Chicago, IL
Kingsland, AR
Locust Ridge, TN
Centre
Bundundu, Zaire
Oakland, CA
London, England
Madrid, Spain
Rio de Janeiro, Brazil
Tennessee
New York, NY [Queens]
New Mexico
Glasgow, Scotland
Fort Wayne, Indiana
Glasgow, Scotland
Kingston, Jamaica
London, England
Long Island, NY
Boston, MA
Gainesville, FL
Chicago
Tennessee
allentown Pennsylvania USA
Tri-Cities, Washington
Haifa, Israel
Bronx,New York City (USA)
Shelby, North Carolina
Stockholm
Denver, CO
Fairfax, California
Paris, France
Port Arthur, TX
London, England
Birmingham, England
East Flatbush, Brooklyn, NY
Atlanta, GA
Duluth, MN
Dundee, Scotland
Chicago
Rome, Italy
Houston, TX
England
Los Angeles, CA
Orange, CA
Dundee, Scotland
Columbia, South Carolina
Fort Myers, FL
Venezuela
St. Joseph, MO
Kingsland, AR
Paddington, London,

Los Angeles, CA
New Jersey
Croix-des-Bouquets,  Haiti
Barranquilla, Colombia
Washington, D.C.
Aberdeen, WA
Regina, Saskatchewan, Canada
Boston, MA
New York City
New Jersey
Minnesota
Muskegon, MI
Oxford, UK
Los Angeles, CA
Los Angeles, CA
London, England
East Flatbush, Brooklyn, NY
Oakland, CA
Bronx, NY
United States
Helsinki
Toyohashi, Japan
The Bronx, New York
Pasadena, CA
Gainesville, FL
Boston, MA
Mascouche, Qc
Philadelphia, PA
McLellan, FL
London, England
Athens, GA
California - SF
England
Galveston, TX
Minnesota
Ukiah, CA
Boston, MA
Oakland CA
Austin
Dublin, Ireland
United States
New York, NY
Long Beach, California
Vrsac
Detroit, MI
Galveston, TX
Cross Lane, WV
Detroit, MI
Medicine Lodge, KS
New Orleans, LA
California
United States
New York
Syracuse, NY
Oxford, UK
Seymour, IN
CHICAGO, Illinois
San Francisco, CA
Los Angeles, CA
Los Angeles, CA
Louisiana
Wisner, LA
CANADA - Ontario
Paddington, London, England
United Kingdom
United Kingdom
Los Angeles, CA
NY - New York City
MILWAUKEE

,genre,genreweight,year,duration,hotttnesss,familiarity,latitude,longitude,location
0,hiphop,6.498262,0,218.93179,0.401998,0.581794,34.052230,-118.243680,California - LA
1,rock,4.344685,1969,148.03546,0.417500,0.630630,35.149680,-90.048920,"Memphis, TN"
2,pop,0.794364,0,177.47546,0.343428,0.487357,NaN,NaN,
3,rock,6.624143,1982,233.40363,0.454231,0.630382,51.506420,-0.127210,"London, England"
4,rock,9.095158,2007,209.60608,0.401724,0.651046,NaN,NaN,
5,blues,1.660828,0,267.70240,0.385471,0.535293,NaN,NaN,
6,hiphop,4.373716,0,114.78159,0.261941,0.556496,40.413057,-82.711215,Ohio
7,rock,1.076524,0,189.57016,0.605507,0.801136,43.326200,-79.798630,"Burlington, Ontario, Canada"
8,rock,5.432989,1984,269.81832,0.332276,0.426668,NaN,NaN,
9,electronica,3.173096,0,266.39628,0.422706,0.550514,NaN,NaN,


In [194]:
df.count() - df.loc[np.isnan(df.longitude)].count()

genre          5687
genreweight    5687
year           5687
duration       5687
hotttnesss     5687
familiarity    5687
latitude       5687
longitude      5687
location       5687
dtype: int64

In [189]:
save_data(df,"df1")

**first look at the genre distribution**

In [ ]:
fig = plt.figure(figsize=(8,6))
df.groupby('genre')['genre'].count().plot('bar',title='Repartition of the different genre')

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
plt.title('cumulative weight')
sns.distplot(df['genreweight'],ax=ax)

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
plt.title('cumulative weight less than <1 (dont keep them?)')
sns.distplot(df['genreweight'][df['genreweight']<1],ax=ax)

**first look at the year distribution**

In [ ]:
print('year=0',len(df[df['year']==0]),'year!=0',len(df[df['year']!=0]))
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
sns.distplot(df['year'][df['year']!=0],ax=ax)

**first look at the hotttness and familiarity distributions**

In [ ]:
print('hotttnesss=0',len(df[df['hotttnesss']==0]))
print('familiarity=0',len(df[df['familiarity']==0]))
df[df['familiarity']==0]

In [ ]:
g = sns.jointplot(x="familiarity", y="hotttnesss", data=df, kind="kde")
g.plot_joint(plt.scatter, s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$familiarity$", "$hotttnesss$");

**first look at the year distribution**

In [ ]:
print('#duration=0 : ',len(df[df['duration']==0]))
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
sns.distplot(df['duration'][df['duration']!=0],ax=ax)

## Understand artist tags
TODO: What's the difference between weights and frequency should we take that into account ?

frequencey is it like frequency on the all database?  rock -> 1

In [ ]:
for root, dirs, files in os.walk(msd_subset_data_path):
    files = glob.glob(os.path.join(root,'*'+'.h5'))
    for f in files:
        h5 = GETTERS.open_h5_file_read(f)
    
        print(GETTERS.get_artist_terms(h5))
        print(GETTERS.get_artist_terms_freq(h5))
        print(GETTERS.get_artist_terms_weight(h5))
        print('')
    
        h5.close()
  

### TODO Geoloc

keep artist name and match location for missing value of same artist ?

find a way to not have this error: `HTTPError: HTTP Error 429: Too Many Requests`

In [ ]:
count_latitude=0
count_location=0

for root, dirs, files in os.walk(msd_subset_data_path):
    files = glob.glob(os.path.join(root,'*'+'.h5'))
    for f in files:
        h5 = GETTERS.open_h5_file_read(f)
    
        if str(GETTERS.get_artist_latitude(h5))!='nan' and str(GETTERS.get_artist_longitude(h5))!='nan' :
            count_latitude+=1

        if GETTERS.get_artist_location(h5)!=b'':
            count_location+=1
            
            
        h5.close()
print(count_latitude,count_location)      

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode('paris')
print(location)

In [ ]:


count_latitude=0

for root, dirs, files in os.walk(msd_subset_data_path):
    files = glob.glob(os.path.join(root,'*'+'.h5'))
    for f in files:
        h5 = GETTERS.open_h5_file_read(f)
    
        if str(GETTERS.get_artist_latitude(h5))!='nan':
            count_latitude+=1
                
            
        else:
            if GETTERS.get_artist_location(h5)!=b'':
                loc=str(GETTERS.get_artist_location(h5))
                
                loc=loc[2:-1]
                try: 
                    location = geolocator.geocode(loc)
                except:
                    print('error',loc)
                    location=None
                if location!=None:
                    count_latitude+=1
                

            
        h5.close()
print(count_latitude,count_location) 

In [ ]:
def getLocation(row):
    if np.isnan(row.lat):
        
        print(row.Location)
        #g = geocoder.google(row.Location)
        #g = geocoder.geonames(row.Location)
        g = geocoder.arcgis(row.Location)
        try:
            lat,lng = g.latlng
        except:
            print("except")
            lat,lng = np.nan,np.nan
            print(lat)
        return pd.Series({'lat':lat,'lng':lng})
    else:
        return pd.Series({'lat':row.lat,'lng':row.lng})

df.loc[:,['lat','lng']] = df.apply(lambda address: getLocation(address),raw=True,axis=1)
df.to_csv('aircrashes1.csv')
df

In [ ]:
from pygeocoder import Geocoder

count_latitude=0

for root, dirs, files in os.walk(msd_subset_data_path):
    files = glob.glob(os.path.join(root,'*'+'.h5'))
    for f in files:
        h5 = GETTERS.open_h5_file_read(f)
    
        if str(GETTERS.get_artist_latitude(h5))!='nan':
            count_latitude+=1
                
            
        else:
            if GETTERS.get_artist_location(h5)!=b'':
                loc=str(GETTERS.get_artist_location(h5))
                
                loc=loc[2:-1]
                try: 
                    valid = Geocoder.geocode(loc).valid_address
                except:
                    print('error', loc)
                    valid='False'
                if valid==True:
                    count_latitude+=1
               
                

            
        h5.close()
print(count_latitude) 